In [2]:
from tools import *
import warnings
warnings.filterwarnings('ignore')

from keras.callbacks import EarlyStopping

from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Prepare the data

In [6]:
# load tabular ukraine data
ukraine_data = pd.read_csv('data/tabular_data_ukraine.csv')

# delete Kyiv and Kyiv_Oblast
# ukraine_data = ukraine_data[ukraine_data["region"] != "Kyiv"]
# ukraine_data = ukraine_data[ukraine_data["region"] != "Kyiv_Oblast"]
ukraine_data = ukraine_data[ukraine_data["region"] != "Kyiv_Oblast_City"]


# get training, test, pre_war and prediction data
train_data = ukraine_data[ukraine_data['year'] < 2021]
test_data = ukraine_data[ukraine_data['year'] == 2021]
pre_war_data = ukraine_data[ukraine_data['year'] < 2022]
prediction_data = ukraine_data[ukraine_data['year'] == 2022]

# reset index
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
pre_war_data = pre_war_data.reset_index(drop=True)
prediction_data = prediction_data.reset_index(drop=True)

# column_prefixes = ("nearnad_snow_cov", "nearnad_snow_free", "offnad_snow_cov",
#                    "offnad_snow_free", "allangle_snow_cov", "allangle_snow_free", 
#                    "nearnad_snow_free_hq", "offnad_snow_free_hq", "allangle_snow_free_hq")

# column_prefixes = ("nearnad_snow_free_hq", "offnad_snow_free_hq", "allangle_snow_free_hq")

column_prefixes = ["allangle_snow_free_hq"]

general_characteristics = ["sum"]

In [7]:
pre_war_data.sort_values(by=['region', 'year'], inplace=True)
pre_war_data_diff = pre_war_data.groupby('region').diff()
pre_war_data_diff['region'] = pre_war_data['region']
pre_war_data_diff['year'] = pre_war_data['year']
pre_war_data_diff.reset_index(drop=True, inplace=True)
pre_war_data_diff = pre_war_data_diff[pre_war_data_diff['year'] != 2012]

prediction_data = pd.concat([ukraine_data[ukraine_data['year'] == 2022], ukraine_data[ukraine_data['year'] == 2021]])
prediction_data.sort_values(by=['region', 'year'], inplace=True)
prediction_data['real_gdp'] = 0
prediction_data_diff = prediction_data.groupby('region').diff()
prediction_data_diff['region'] = prediction_data['region']
prediction_data_diff['year'] = prediction_data['year']
prediction_data_diff.reset_index(drop=True, inplace=True)
prediction_data_diff = prediction_data_diff[prediction_data_diff['year'] == 2022]

Define general functions

XGBoost

In [8]:
# Define parameter grid for XGBoost
param_grid_xgb = {
    'eta': [0.1, 0.2, 0.3, 0.4],
    'gamma': [0, 5, 10, 20],
    'max_depth': [4, 6, 8, 10],
    'min_child_weight': [3, 4, 5, 6],
    'random_state': [0] 
}

In [9]:
# initialise a df to store the results
xgb_results = pd.DataFrame(columns=["prefix", "columns", "mae", "mpe", "national_gdp_change"])

for prefix in column_prefixes:
    # create general column names
    general_columns, log_bin_columns, idr_bin_columns = create_column_names(prefix, general_characteristics)
    
    # build xgb models for each year, calculate average mpe and mse, predict the national gdp change, add the results to the df
    for selected_columns, columns_category in zip([log_bin_columns], ["log_bin"]):
        mae, mpe, gdp_change, y_pred, best_params, metrics = build_model_and_predict(pre_war_data_diff, prediction_data_diff, selected_columns, "xgboost", param_grid_xgb, log_transform = False, scale = False, total_metrics = True, diff = True)
        new_results = pd.DataFrame([{"prefix": prefix, "columns": columns_category, "mae": mae, "mpe": mpe, "national_gdp_change": gdp_change}])
        xgb_results = pd.concat([xgb_results, new_results], ignore_index=True)

    print(f"Finished {prefix}")

# save metrics as a csv
# metrics.to_csv("xgb_results.csv", index=False)

Finished allangle_snow_free_hq


In [12]:
metrics["mae"].mean()

2871.5517195111765

Random Forest

In [9]:
# Define parameter grid for Random Forest
param_grid_rf = {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
        'random_state': [0],  # Ensures reproducibility

}

In [10]:
# initialise a df to store the results
rf_results = pd.DataFrame(columns=["prefix", "columns", "mae", "mpe", "national_gdp_change"])

for prefix in column_prefixes:
    # create general column names
    general_columns, log_bin_columns, idr_bin_columns = create_column_names(prefix, general_characteristics)
    
    # build xgb models and predict the national gdp change, add the results to the df
    for selected_columns, columns_category in zip([log_bin_columns], ["log_bin"]):
        mae, mpe, gdp_change, y_pred, best_params, metrics = build_model_and_predict(pre_war_data_diff, prediction_data_diff, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False, total_metrics = True, diff = True)
        new_results = pd.DataFrame([{"prefix": prefix, "columns": columns_category, "mae": mae, "mpe": mpe, "national_gdp_change": gdp_change}])
        rf_results = pd.concat([rf_results, new_results], ignore_index=True)

    print(f"Finished {prefix}")

# save metrics as a csv
metrics.to_csv("rf_results.csv", index=False)

Finished allangle_snow_free_hq


In [37]:
# initialise a df to store the results
nn_results = pd.DataFrame(columns = ["mae", "year"])
log_bin_columns = ["allangle_snow_free_hq" + "_log_" + str(i) for i in range(1, 11)]
set_seed(1)

for test_year in range(2013, 2022):
    # calculate differences
    ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
    ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
    ukraine_sum = ukraine[['year', 'region', 'real_gdp', 'allangle_snow_free_hq_sum'] + log_bin_columns]
    ukraine = ukraine[ukraine["year"] < 2022]

    full_data = ukraine_sum
    full_data.sort_values(by=['region', 'year'], inplace=True)
    full_data_diff = full_data.groupby('region').diff()
    full_data_diff['region'] = full_data['region']
    full_data_diff['year'] = full_data['year']
    full_data_diff.reset_index(drop=True, inplace=True)
    full_data_diff.dropna(inplace=True)

    train_data_diff = full_data_diff[full_data_diff["year"] != test_year]
    test_data_diff = full_data_diff[full_data_diff["year"] == test_year]
    test_data_diff.reset_index(drop=True, inplace=True)
    train_data_diff.reset_index(drop=True, inplace=True)

    # build the model
    # mae, _, _ = build_model(train_data_diff, test_data_diff, selected_columns, "random_forest", param_grid_rf, 
    #                                                                             log_transform = False, scale = False)

    train_data_nn = train_data_diff.copy()
    pred_data_nn = test_data_diff.copy()

    train_data_nn = pd.get_dummies(train_data_nn, columns=["region"])
    pred_data_nn = pd.get_dummies(pred_data_nn, columns=["region"])

    X_train_nn = train_data_nn.drop(columns=["year", "real_gdp"])
    y_train_nn = train_data_nn["real_gdp"]

    X_test_nn = pred_data_nn.drop(columns=["year", "real_gdp"])
    y_test_nn = pred_data_nn["real_gdp"]

    X_train_nn = np.array(X_train_nn, dtype=np.float32)
    X_test_nn = np.array(X_test_nn, dtype=np.float32)
    y_train_nn = np.array(y_train_nn, dtype=np.float32)
    y_test_nn = np.array(y_test_nn, dtype=np.float32)


    # scale the data
    scaler = StandardScaler()
    X_train_nn = scaler.fit_transform(X_train_nn)
    X_test_nn = scaler.transform(X_test_nn)

    # fit the model on the X_train and y_train
    model = Sequential()
    # model.add(Dense(256, activation="relu", input_dim=X_train_nn.shape[1]))  # First layer
    # model.add(Dense(128, activation="relu"))  # First layer
    # model.add(Dense(64, activation="relu"))  # First layer
    model.add(Dense(32, activation="relu", input_dim=X_train_nn.shape[1]))  # Fourth layer
    model.add(Dense(16, activation="relu"))  # New additional layer
    model.add(Dense(8, activation="relu"))  # New additional layer
    model.add(Dense(1, activation = 'linear')) 

    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])


    early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    model.fit(X_train_nn, y_train_nn, epochs=500, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

    y_pred = model.predict(X_test_nn).flatten()
    mae = np.mean(np.abs(y_pred - y_test_nn))

    # add the results to the dataframe
    new_row = pd.DataFrame([{
        "mae": mae,
        "year": test_year
    }])

    nn_results = pd.concat([nn_results, new_row], ignore_index=True)
    print("Results for year = ", test_year)

Epoch 1/500
3/3 [==============================] - 1s 153ms/step - loss: 47124960.0000 - mae: 2917.2324 - val_loss: 2102498.5000 - val_mae: 1221.5280
Epoch 2/500
3/3 [==============================] - 0s 23ms/step - loss: 47124792.0000 - mae: 2917.2212 - val_loss: 2102492.5000 - val_mae: 1221.5271
Epoch 3/500
3/3 [==============================] - 0s 23ms/step - loss: 47124664.0000 - mae: 2917.2134 - val_loss: 2102487.7500 - val_mae: 1221.5260
Epoch 4/500
3/3 [==============================] - 0s 22ms/step - loss: 47124552.0000 - mae: 2917.2056 - val_loss: 2102484.0000 - val_mae: 1221.5247
Epoch 5/500
3/3 [==============================] - 0s 23ms/step - loss: 47124456.0000 - mae: 2917.1968 - val_loss: 2102480.5000 - val_mae: 1221.5231
Epoch 6/500
3/3 [==============================] - 0s 25ms/step - loss: 47124360.0000 - mae: 2917.1880 - val_loss: 2102476.7500 - val_mae: 1221.5212
Epoch 7/500
3/3 [==============================] - 0s 24ms/step - loss: 47124232.0000 - mae: 2917.1782 - 

In [41]:
# nn_results.to_csv("nn_results_aa_sf_hq.csv", index=False)
print(nn_results["mae"].mean())


2631.9602


In [42]:
nn_results

,mae,year
0,2284.263672,2013
1,3435.992188,2014
2,4568.526367,2015
3,2481.959961,2016
4,1909.524658,2017
5,3141.903320,2018
6,1757.951294,2019
7,2198.557861,2020
8,1908.963745,2021


In [77]:
# predict
log_bin_columns = ["nearnad_snow_cov" + "_log_" + str(i) for i in range(1, 11)]
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
# ukraine = pd.read_csv("data/tabular_data_poland.csv")
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
ukraine_sum = ukraine[['year', 'region', 'real_gdp', 'nearnad_snow_cov_sum'] + log_bin_columns]
ukraine_sum = ukraine_sum[ukraine_sum["year"] < 2023]

full_data = ukraine_sum
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
train_data_diff.dropna(inplace=True)
test_data_diff = full_data_diff[full_data_diff["year"] == 2022]
test_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)

# build the model
# mae, _, _ = build_model(train_data_diff, test_data_diff, selected_columns, "random_forest", param_grid_rf, 
#                                                                             log_transform = False, scale = False)

train_data_nn = train_data_diff.copy()
pred_data_nn = test_data_diff.copy()

train_data_nn = pd.get_dummies(train_data_nn, columns=["region"])
pred_data_nn = pd.get_dummies(pred_data_nn, columns=["region"])

X_train_nn = train_data_nn.drop(columns=["year", "real_gdp"])
y_train_nn = train_data_nn["real_gdp"]

X_test_nn = pred_data_nn.drop(columns=["year", "real_gdp"])
y_test_nn = pred_data_nn["real_gdp"]

X_train_nn = np.array(X_train_nn, dtype=np.float32)
X_test_nn = np.array(X_test_nn, dtype=np.float32)
y_train_nn = np.array(y_train_nn, dtype=np.float32)
y_test_nn = np.array(y_test_nn, dtype=np.float32)


# scale the data
scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train_nn)
X_test_nn = scaler.transform(X_test_nn)
set_seed(1) 
# fit the model on the X_train and y_train
model = Sequential()
# model.add(Dense(256, activation="relu", input_dim=X_train_nn.shape[1]))  # First layer
# model.add(Dense(128, activation="relu"))  # First layer
# model.add(Dense(64, activation="relu"))  # First layer
model.add(Dense(32, activation="relu", input_dim=X_train_nn.shape[1]))  # Fourth layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(8, activation="relu"))  # New additional layer
model.add(Dense(1, activation = 'linear')) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"], )


# add early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)
model.fit(X_train_nn, y_train_nn, epochs=200, batch_size=64, validation_split=0.2)

y_pred = model.predict(X_test_nn).flatten()


Epoch 1/200
3/3 [==============================] - 1s 92ms/step - loss: 44507780.0000 - mae: 2893.3135 - val_loss: 1933771.2500 - val_mae: 1146.0251
Epoch 2/200
3/3 [==============================] - 0s 20ms/step - loss: 44507576.0000 - mae: 2893.3020 - val_loss: 1933781.8750 - val_mae: 1146.0286
Epoch 3/200
3/3 [==============================] - 0s 20ms/step - loss: 44507412.0000 - mae: 2893.2954 - val_loss: 1933791.2500 - val_mae: 1146.0308
Epoch 4/200
3/3 [==============================] - 0s 18ms/step - loss: 44507304.0000 - mae: 2893.2883 - val_loss: 1933798.2500 - val_mae: 1146.0322
Epoch 5/200
3/3 [==============================] - 0s 19ms/step - loss: 44507208.0000 - mae: 2893.2837 - val_loss: 1933803.7500 - val_mae: 1146.0331
Epoch 6/200
3/3 [==============================] - 0s 19ms/step - loss: 44507096.0000 - mae: 2893.2791 - val_loss: 1933808.8750 - val_mae: 1146.0336
Epoch 7/200
3/3 [==============================] - 0s 18ms/step - loss: 44507024.0000 - mae: 2893.2769 - v

In [78]:
sum(y_pred)/ukraine_sum[ukraine_sum["year"] == 2021]["real_gdp"].sum()

-0.08447469647039074

In [79]:

test_data_diff["gdp_pred"] = y_pred
ukraine_2022_pred = test_data_diff[["region", "gdp_pred"]]
ukraine_2021 = ukraine_sum[ukraine_sum["year"] == 2021]
ukraine_2022_pred = pd.merge(ukraine_2022_pred, ukraine_2021[["region", "real_gdp"]], on=["region"])
ukraine_2022_pred["gdp_pred"] = ukraine_2022_pred["gdp_pred"] + ukraine_2022_pred["real_gdp"]
ukraine_2022_pred = ukraine_2022_pred[["region", "gdp_pred"]]

# save the results
ukraine_2022_pred.to_csv("gdp_predictions_ukraine_nn_sc_lq.csv", index=False)

In [21]:
ukraine_2022_pred

,region,gdp_pred
0,Cherkasy_Oblast,25683.734003
1,Chernihiv_Oblast,17936.880791
2,Chernivtsi_Oblast,6675.608835
3,Dnipropetrovsk_Oblast,58510.873612
4,Donetsk_Oblast,33712.934306
5,Ivano-Frankivsk_Oblast,21746.145738
6,Kharkiv_Oblast,45717.755352
7,Kherson_Oblast,16179.705971
8,Khmelnytskyi_Oblast,23206.423099
9,Kirovohrad_Oblast,17040.693394


In [19]:
test_data_diff["gdp_pred"] = y_pred
poland_2022_pred = test_data_diff[["region", "gdp_pred", "real_gdp"]]
poland_2022_pred.to_csv("gdp_predictions_poland_nn.csv", index=False)   